In [1]:
# TODO(deflaux): remove this cell after gcr.io/broad-ml4cvd/deeplearning:tf2-latest-gpu has this preinstalled.
from ml4h.runtime_data_defines import determine_runtime
from ml4h.runtime_data_defines import Runtime

if Runtime.ML4H_VM == determine_runtime():
  !pip3 install --user ipycanvas==0.7.0 ipyannotations==0.2.1
  !jupyter nbextension install --user --py ipycanvas
  !jupyter nbextension enable --user --py ipycanvas
  # Be sure to restart the kernel if pip installs anything.
  # Also, shift-reload the browser page after the notebook extension installation.

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Installing /root/.local/lib/python3.6/site-packages/ipycanvas/nbextension/static -> ipycanvas
Up to date: /root/.local/share/jupyter/nbextensions/ipycanvas/index.js
Up to date: /root/.local/share/jupyter/nbextensions/ipycanvas/extension.js
Up to date: /root/.local/share/jupyter/nbextensions/ipycanvas/index.js.map
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable ipycanvas --user --py
    
Enabling notebook extension ipycanvas/extension...
      - Validating: OK


In [2]:
from ipyannotations import BoxAnnotator, PointAnnotator, PolygonAnnotator
from ml4h.visualization_tools.annotation_storage import DataFrameAnnotationStorage
from ml4h.visualization_tools.batch_image_annotations import BatchImageAnnotator
import pandas as pd
import tensorflow as tf

In [3]:
%%javascript
// Display cell outputs to full height (no vertical scroll bar)
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


# Define the batch of samples to annotate

<div class="alert alert-block alert-info">
    Edit the CSV file path below, if needed, to either a local file or one in Cloud Storage.
</div>

In [5]:
#---[ EDIT AND RUN THIS CELL TO READ FROM A LOCAL FILE OR A FILE IN CLOUD STORAGE ]---
import glob

pngs = glob.glob('/home/pdiachil/projects/t1map/inference/inference_output/*.png')
png_dic = {'sample_id': [], 'tmap_name': [], 'instance_number': [], 'folder': []}
for png in pngs:
    sample = png.split('_')[-4]
    instance = png.split('_')[-3]
    png_dic['sample_id'].append(int(sample))
    png_dic['instance_number'].append(int(instance))
    png_dic['folder'].append('/home/pdiachil/projects/t1map/inference/inference_output')
    png_dic['tmap_name'].append('segmented')

In [6]:
png_df = pd.DataFrame(png_dic)
png_df

,sample_id,tmap_name,instance_number,folder
0,1005225,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
1,4517442,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
2,1471500,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
3,4439172,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
4,3999707,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
...,...,...,...,...
45477,2404220,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
45478,4527197,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
45479,3316061,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
45480,1556312,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output


In [14]:
png_df_shuffle = png_df.sample(frac=1)
png_df_shuffle

,sample_id,tmap_name,instance_number,folder
29491,4754055,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
37943,3677123,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
17958,2342832,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
28540,4775078,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
36440,1176560,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
...,...,...,...,...
32079,2332377,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
24269,5643713,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
20551,1576898,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
13670,1216330,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output


In [18]:
block_size = 45482 // 4
for i, user in enumerate(['pdiachil', 'mklarqvist', 'vnauffal', 'jcunning']):
    png_df_shuffle.iloc[i*block_size : (i+1)*block_size].to_csv(f'png_split_{user}.csv', index=False)

In [17]:
png_df_shuffle.iloc[i*block_size : (i+1)*block_size]

,sample_id,tmap_name,instance_number,folder
33189,5539400,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
31984,3994967,segmented,3,/home/pdiachil/projects/t1map/inference/inference_output
33784,3863645,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
5022,2881791,segmented,3,/home/pdiachil/projects/t1map/inference/inference_output
20184,2217876,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
...,...,...,...,...
1420,4921826,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
41268,3073893,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
32079,2332377,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output
24269,5643713,segmented,2,/home/pdiachil/projects/t1map/inference/inference_output


# Annotate the batch! 

In [12]:
# Note: a zoom level of 1.0 displays the tensor as-is. For higher zoom levels, this code currently
# use the PIL library to scale the image.
import datetime

output_filename = f'goodbad_t1map_inference_pdiachil_{datetime.datetime.now().strftime("%m_%d_%y_%H_%M_%S")}.csv'

annotator = BatchImageAnnotator(samples=png_df.head(10),
                                zoom=1.5,
                                annotation_categories=['good', 'bad', 'review'],
                                annotation_storage=DataFrameAnnotationStorage(output_filename),
                                annotator=PointAnnotator,
                                user='pdiachil'
                               )
annotator.annotate_images()

/home/pdiachil/projects/t1map/inference/inference_output/*1005225*2*png


/home/pdiachil/projects/t1map/inference/inference_output/*4517442*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*1471500*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*4439172*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*3999707*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*3709141*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*3020689*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*1286390*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*1126305*2*png
/home/pdiachil/projects/t1map/inference/inference_output/*5663218*2*png


## Annotate with rectangles

Use rectangles to annotate rectangular regions within the image.

# View the stored annotations 

In [8]:
annotator.view_recent_submissions(count=10)

""


# Provenance

In [9]:
import datetime
print(datetime.datetime.now())

2021-05-17 10:52:40.483141


In [10]:
%%bash
pip3 freeze

absl-py==0.9.0
altair==4.1.0
apache-beam==2.12.0
asn1crypto==0.24.0
astor==0.8.1
attrs==19.3.0
avro-python3==1.9.2.1
backcall==0.1.0
beautifulsoup4==4.9.0
bidict==0.19.0
biosppy==0.6.1
bleach==3.1.5
blosc==1.10.2
bokeh==2.0.2
cachetools==4.0.0
certifi==2019.11.28
chardet==3.0.4
cloudpickle==1.4.1
coverage==5.1
crcmod==1.7
cryptography==2.1.4
cvxopt==1.2.5
cvxpy==1.0.31
cycler==0.10.0
Cython==0.29.17
dataclasses==0.8
decorator==4.4.2
defusedxml==0.6.0
descartes==1.1.0
dill==0.2.9
distro-info===0.18ubuntu0.18.04.1
docopt==0.6.2
ecos==2.0.7.post1
entrypoints==0.3
facets-overview==1.0.0
fastavro==0.21.24
fasteners==0.15
future==0.18.2
gast==0.2.2
google-api-core==1.17.0
google-apitools==0.5.26
google-auth==1.14.3
google-auth-oauthlib==0.4.1
google-cloud-bigquery==1.6.2
google-cloud-bigquery-storage==2.4.0
google-cloud-bigtable==0.31.1
google-cloud-core==0.28.1
google-cloud-pubsub==0.39.0
google-cloud-storage==1.13.0
google-pasta==0.1.8
google-resumable-media==0.4.1
googleapis-common-protos

ERROR: Error [Errno 2] No such file or directory: 'git': 'git' while executing command git config --get-regexp 'remote\..*\.url'


In [11]:
annotator.annotation_widget.data

[]

Questions about these particular notebooks? Join the discussion https://github.com/broadinstitute/ml4h/discussions.